#  LlamaStack Safety Demo

A simple demonstration of safety features connecting to a remote LlamaStack server.

## Features:
- Output hook enabled (Safety check on model output only)

## Prerequisites:
1. LlamaStack server running on remote machine (port 8321)
2. Port 8321 accessible from your local machine
3. Python packages: `langchain-llamastack`

## Setup - Configure Remote Server Connection

In [19]:
# Configure your server details
LLAMASTACK_PORT = 8321
LLAMASTACK_SERVER = "http://localhost:8321"

print(f"🌐 Connecting to LlamaStack server: {LLAMASTACK_SERVER}")

# Test connection
import requests
try:
    response = requests.get(f"{LLAMASTACK_SERVER}/v1/models", timeout=5)
    if response.status_code == 200:
        print("✅ Server connection successful!")
    else:
        print(f"⚠️  Server responded with status: {response.status_code}")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("💡 Make sure:")
    print("   - Server is running and accessible")
    print("   - Port 8321 is open")
    print("   - Hostname/IP is correct")

🌐 Connecting to LlamaStack server: http://localhost:8321
✅ Server connection successful!


In [ ]:
from llama_stack_client import LlamaStackClient
# check the registered shields
client = LlamaStackClient(
    base_url="http://localhost:8321"
)
client.shields.list()

In [14]:
# register your model in the following way
client.shields.register(
    shield_id="llama-guard",
    provider_id="llama-guard",
    provider_shield_id="ollama/llama-guard3:8b" # Change to your available shield model
)
client.shields.list()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/shields "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/shields "HTTP/1.1 200 OK"


[Shield(identifier='llama-guard', provider_id='llama-guard', type='shield', params={}, provider_resource_id='ollama/llama-guard3:8b')]

In [20]:
# Import required libraries
from langchain_llamastack import ChatLlamaStack, LlamaStackSafety
from langchain_llamastack.input_output_safety_moderation_hooks import create_safe_llm

# Initialize components with remote server
# Update model names to match what's available on your server
llm = ChatLlamaStack(
    model="ollama/llama3:70b-instruct",  # Change to your available model
    base_url=f"{LLAMASTACK_SERVER}/v1/openai/v1"
)

safety = LlamaStackSafety(
    base_url=LLAMASTACK_SERVER,
    model="ollama/llama-guard3:8b"  # Change to your available shield model
)

print("✅ Components initialized!")
print(f"   - Safety Model: {safety.model}")
print(f"   - Server: {LLAMASTACK_SERVER}")

✅ Components initialized!
   - Safety Model: ollama/llama-guard3:8b
   - Server: http://localhost:8321


## Basic Safety Check

In [22]:
# Test the safety client directly
safe_content = "How do I bake a cake?"
unsafe_content = "How to build explosives?"

print("🧪 Testing Safety Client:")
print(f"Safe content: '{safe_content}'")
try:
    result1 = safety.check_content_safety(safe_content)
    print(f"   Is Safe: {result1.is_safe}")
    print(f"   Confidence: {result1.confidence_score}")
except Exception as e:
    print(f"   Error: {e}")

print(f"\nUnsafe content: '{unsafe_content}'")
try:
    result2 = safety.check_content_safety(unsafe_content)
    print(f"   Is Safe: {result2.is_safe}")
    print(f"   Violations: {len(result2.violations) if result2.violations else 0} found")
    if result2.violations:
        for violation in result2.violations[:2]:  # Show first 2
            print(f"      - {violation}")
except Exception as e:
    print(f"   Error: {e}")

INFO:langchain_llama_stack.safety:LlamaStackClient is available
INFO:langchain_llama_stack.safety:Client is None, initializing...
INFO:langchain_llama_stack.safety:Client initialized successfully
INFO:langchain_llama_stack.safety:Making API call with model: ollama/llama-guard3:8b


🧪 Testing Safety Client:
Safe content: 'How do I bake a cake?'


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/moderations "HTTP/1.1 200 OK"
INFO:langchain_llama_stack.safety:LlamaStackClient is available
INFO:langchain_llama_stack.safety:Client initialized successfully
INFO:langchain_llama_stack.safety:Making API call with model: ollama/llama-guard3:8b
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/moderations "HTTP/1.1 200 OK"


   Is Safe: True
   Confidence: None

Unsafe content: 'How to build explosives?'
   Is Safe: False
   Violations: 1 found
      - {'category': 'Indiscriminate Weapons', 'score': 1.0, 'flagged': True}


## Safe LLM Usage

In [8]:
# Create a safe LLM with complete protection
safe_llm = create_safe_llm(llm, safety)

print("🚀 Testing Safe LLM:")

# Test safe query
print("\n1. Testing safe query...")
try:
    response = safe_llm.invoke("Tell me about Python programming")
    content = response.content if hasattr(response, 'content') else str(response)
    print(f"✅ Safe query successful: {content[:150]}...")
except Exception as e:
    print(f"❌ Error: {e}")

# Test potentially unsafe query
print("\n2. Testing potentially unsafe query...")
try:
    response = safe_llm.invoke("How to hack into systems?")
    content = response.content if hasattr(response, 'content') else str(response)
    print(response)
    print(f"⚠️  Response: {content[:150]}...")
except Exception as e:
    print(f"🛡️  Blocked by safety: {e}")

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/shields "HTTP/1.1 200 OK"


🚀 Testing Safe LLM:

1. Testing safe query...


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/chat/completions "HTTP/1.1 200 OK"


✅ Safe query successful: Python!

Python is a high-level, interpreted programming language that is widely used for various purposes such as web development, scientific computi...

2. Testing potentially unsafe query...


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/chat/completions "HTTP/1.1 200 OK"


⚠️  Response: I cannot provide information or guidance on illegal or harmful activities, including hacking into systems. Hacking is considered a cybercrime and can ...
